In [ ]:
import calendar
import folium
import matplotlib.cm as cm
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from windrose import WindroseAxes, plot_windrose


In [ ]:
def remove_rows_with_missing_values(df, column1, column2=None):
    columns_to_check = [column1]
    if column2:
        columns_to_check.append(column2)
    return df.dropna(subset=columns_to_check)


In [ ]:
def plot_windrose_subplots(data, *, direction, var, color=None, **kwargs):
    """wrapper function to create subplots per axis"""
    ax = plt.gca()
    ax = WindroseAxes.from_ax(ax=ax)
    plot_windrose(direction_or_df=data[direction], var=data[var], ax=ax, **kwargs)


In [ ]:
def plot_kde_facetgrid(
    df,
    magnitude_column="average_wind_speed",
    title=None,
    base_path=None,
    subfolder=None,
):

    # Assuming remove_rows_with_missing_values and column_labels are defined elsewhere
    df = remove_rows_with_missing_values(df, magnitude_column)
    df = df[df.index.year != 2024]

    df["Year"] = df.index.year
    df["Month"] = df.index.month
    df["MonthName"] = df["Month"].apply(lambda x: calendar.month_abbr[x])

    # Set xlabel based on magnitude_column using the column_labels dictionary
    xlabel = column_labels.get(magnitude_column, magnitude_column)

    if base_path and subfolder:
        full_path = os.path.join(base_path, subfolder)
        os.makedirs(full_path, exist_ok=True)
    else:
        full_path = base_path

    g_kde = sns.displot(
        data=df,
        x=magnitude_column,
        col="Year",
        hue="MonthName",
        kind="kde",
        col_wrap=3,
        height=3.5,
        facet_kws={"sharex": False, "sharey": False},
        common_norm=False,
        warn_singular=False,
    )
    g_kde.set_xlabels(xlabel)
    g_kde._legend.set_title("Month")

    # Adjusting the position of the legend
    # plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

    plt.tight_layout(rect=[0, 0.03, 0.9, 0.97])
    if title:
        plt.gcf().suptitle(title, fontsize=16, y=0.98)

    if full_path:
        kde_save_path = os.path.join(full_path, f"{title}_kde.svg")
        plt.savefig(kde_save_path, format="svg")


In [ ]:
def plot_ecdf_facetgrid(
    df,
    magnitude_column="average_wind_speed",
    title=None,
    base_path=None,
    subfolder=None,
):
    df = remove_rows_with_missing_values(df, magnitude_column)
    df = df[df.index.year != 2024]

    df["Year"] = df.index.year
    df["Month"] = df.index.month
    df["MonthName"] = df["Month"].apply(lambda x: calendar.month_abbr[x])

    # Set xlabel based on magnitude_column using the column_labels dictionary
    xlabel = column_labels.get(magnitude_column, magnitude_column)

    if base_path and subfolder:
        full_path = os.path.join(base_path, subfolder)
        os.makedirs(full_path, exist_ok=True)
    else:
        full_path = base_path

    g_ecdf = sns.displot(
        data=df,
        x=magnitude_column,
        col="Year",
        hue="MonthName",
        kind="ecdf",
        col_wrap=3,
        height=3.5,
        facet_kws={"sharex": False, "sharey": False},
    )
    g_ecdf.set_xlabels(xlabel)
    g_ecdf._legend.set_title("Month")
    plt.tight_layout(rect=[0, 0.03, 0.9, 0.97])
    if title:
        plt.gcf().suptitle(title, fontsize=16, y=0.98)

    if title:
        ecdf_save_path = os.path.join(full_path, f"{title}_ecdf.svg")
        plt.savefig(ecdf_save_path, format="svg")


In [ ]:
def plot_timeseries_facetgrid(
    df, magnitude_column=None, title=None, base_path=None, subfolder=None
):
    df = remove_rows_with_missing_values(df, magnitude_column).copy()

    df = df[df.index.year != 2024]

    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    df["Year"] = df.index.year
    years = df["Year"].unique()
    num_years = len(years)
    fig, axs = plt.subplots(num_years, 1, figsize=(15, 4 * num_years), sharex=False)

    if num_years == 1:
        axs = [axs]

    # Set ylabel based on magnitude_column using the column_labels dictionary
    ylabel = column_labels.get(magnitude_column, magnitude_column)

    for i, year in enumerate(years):
        year_data = df[df["Year"] == year]
        axs[i].plot(year_data.index, year_data[magnitude_column])
        axs[i].set_title(f"Year: {year}", fontsize=14)

        # Set x-axis limits and other plot configurations
        start_of_year = pd.Timestamp(year=year, month=1, day=1)
        end_of_year = pd.Timestamp(year=year, month=12, day=31)
        axs[i].set_xlim(start_of_year, end_of_year)

        axs[i].xaxis.set_major_locator(mdates.MonthLocator())
        axs[i].xaxis.set_major_formatter(ticker.NullFormatter())
        axs[i].xaxis.set_minor_locator(mdates.MonthLocator(bymonthday=16))
        axs[i].xaxis.set_minor_formatter(mdates.DateFormatter("%b"))
        axs[i].tick_params(axis="x", which="minor", tick1On=False, tick2On=False)
        for label in axs[i].get_xticklabels(minor=True):
            label.set_horizontalalignment("center")

        plt.setp(axs[i].xaxis.get_majorticklabels(), rotation=0, fontsize=12)
        axs[i].set_ylabel(ylabel, fontsize=12)

    plt.tight_layout(rect=[0, 0.03, 1, 0.97])

    if title:
        fig.suptitle(title, fontsize=16, y=0.98)

    if base_path and subfolder:
        full_path = os.path.join(base_path, subfolder)
        os.makedirs(full_path, exist_ok=True)
        save_path = os.path.join(full_path, f"{title}_timeseries_by_year.svg")
        plt.savefig(save_path, format="svg")


In [ ]:
def plot_kde_facetgrid(
    df, magnitude_column=None, title=None, base_path=None, subfolder=None
):

    # Assuming remove_rows_with_missing_values and column_labels are defined elsewhere
    df = remove_rows_with_missing_values(df, magnitude_column)
    df = df[df.index.year != 2024]

    df["Year"] = df.index.year
    df["Month"] = df.index.month
    df["MonthName"] = df["Month"].apply(lambda x: calendar.month_abbr[x])

    # Set xlabel based on magnitude_column using the column_labels dictionary
    xlabel = column_labels.get(magnitude_column, magnitude_column)

    if base_path and subfolder:
        full_path = os.path.join(base_path, subfolder)
        os.makedirs(full_path, exist_ok=True)
    else:
        full_path = base_path

    g_kde = sns.displot(
        data=df,
        x=magnitude_column,
        col="Year",
        hue="MonthName",
        kind="kde",
        col_wrap=3,
        height=3.5,
        facet_kws={"sharex": False, "sharey": False},
        common_norm=False,
        warn_singular=False,
    )
    g_kde.set_xlabels(xlabel)
    g_kde._legend.set_title("Month")

    # Adjusting the position of the legend
    # plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

    plt.tight_layout(rect=[0, 0.03, 0.9, 0.97])
    if title:
        plt.gcf().suptitle(title, fontsize=16, y=0.98)

    if full_path:
        kde_save_path = os.path.join(full_path, f"{title}_kde.svg")
        plt.savefig(kde_save_path, format="svg")


In [8]:
def plot_kde(
                df, 
                direction_column, 
                magnitude_column, 
                year, 
                title, 
                base_path=None, 
                subfolder=None
                ):
    if base_path and subfolder:
        full_path = os.path.join(base_path, subfolder)
        os.makedirs(full_path, exist_ok=True)
    else:
        full_path = base_path

    df = df.copy()
    orinetation = 90 - 34
    df[direction_column] = (df[direction_column] - orinetation) % 360
    df.index = pd.to_datetime(df.index)
    df_year = df[df.index.year == year]

    # for month in range(1, 13):
    #     df_month = df_year[df_year.index.month == month]
    #     df_sliced = df_month[df_month[magnitude_column] > 1]

    df_sliced = df
    sns.set_theme(style="ticks")
    g = sns.JointGrid(
        data=df_sliced,
        x=direction_column,
        y=magnitude_column,
        marginal_ticks=True,
        space=0.5
    )
    g.fig.set_facecolor('white')
    cbar_ax = g.fig.add_axes([0.63, 0.15, 0.05, 0.2])  # x, y, width, height

    g.plot_joint(
        sns.kdeplot,
        log_scale=(False, True),
        fill=True,
        thresh=0,
        cut=0,
        levels=10,
        cbar=True,
        cbar_ax=cbar_ax
    )
    cbar_ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x:.3f}'))

    g.plot_marginals(
        sns.histplot,
        alpha=0.5,
        stat="probability",
        kde=True,
        bins=10,
        color="#03012d",
        element="step"
    )

    # Set y-axis label for the marginal histograms
    g.ax_marg_x.set_ylabel("Probability")
    g.ax_marg_y.set_ylabel("Probability")

    if magnitude_column == 'wave_height':
        ylabel = r"Wave Height (m)"
        xlabel = "Wave Direction (degree)"
        MultipleLocator = 1
    elif magnitude_column in ['wind_speed', 'average_wind_speed']:
        ylabel = r"Wind Speed (m/s)"
        xlabel = "Wind Direction (degree)"
        MultipleLocator = 2

    g.set_axis_labels(f"{xlabel}", f"{ylabel}")
    g.ax_joint.yaxis.set_major_locator(ticker.MultipleLocator(MultipleLocator))
    g.ax_joint.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: '{:.0f}'.format(y)))
    g.ax_joint.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '{:.0f}'.format(x)))
    g.ax_joint.xaxis.set_major_locator(ticker.FixedLocator([0, 90, 180, 270, 360]))

    cm = 1/2.54  # centimeters in inches
    plt.gcf().set_size_inches(20*cm, 20*cm)  # Size in cm

    plt.suptitle(f'{title}', horizontalalignment='center', verticalalignment='top')
    
    
    # plt.suptitle(f'{title}\nYear: {year} - Month: {month}', horizontalalignment='center', verticalalignment='top')
    
    plt.tight_layout()
    
    save_path = os.path.join(full_path, f"{title}.svg") 
    
    # save_path = os.path.join(full_path, f"{title},Year {year},Month {month}.svg") if full_path else f"{title},Year {year},Month {month}.svg"
    plt.savefig(save_path, format='svg')
    plt.close(g.fig)

In [ ]:
def plot_windrose_facetgrid(
    df,
    direction_column="wind_direction",
    magnitude_column="wind_speed",
    base_path=None,
    subfolder=None,
    title=None,
):
    df = remove_rows_with_missing_values(df, direction_column, magnitude_column)
    df["Year"] = df.index.year
    df["Month"] = df.index.month
    if base_path and subfolder:
        full_path = os.path.join(base_path, subfolder)
        os.makedirs(full_path, exist_ok=True)
    else:
        full_path = base_path

    g = sns.FacetGrid(
        data=df,
        col="Year",
        col_wrap=3,
        subplot_kws={"projection": "windrose"},
        sharex=False,
        sharey=False,
        despine=False,
        height=3.5,
    )

    if magnitude_column == "wave_height":
        bins = (0.1, 1, 2, 3, 4, 5)
    elif magnitude_column in ["wind_speed", "average_wind_speed"]:
        bins = (0.1, 3, 6, 9, 12, 15)

    g.map_dataframe(
        plot_windrose_subplots,
        direction=direction_column,
        var=magnitude_column,
        normed=True,
        bins=bins,
        calm_limit=0.1,
        kind="bar",
    )

    y_ticks = range(0, 17, 4)
    if magnitude_column == "wave_height":
        legend_title = "$m$"
    elif magnitude_column in ["wind_speed", "average_wind_speed"]:
        legend_title = "$m \cdot s^{-1}$"
    for ax in g.axes:
        ax.set_legend(
            title=legend_title, bbox_to_anchor=(1.15, -0.1), loc="lower right"
        )
        ax.set_rgrids(y_ticks, y_ticks)

    plt.subplots_adjust(wspace=-0.2, top=0.9)
    if title:
        plt.suptitle(title, fontsize=16)

    if full_path:
        save_path = os.path.join(full_path, f"{title}_windrose.svg")
        plt.savefig(save_path, format="svg")

    sns.displot(
        data=df,
        x=magnitude_column,
        col="Year",
        hue="Month",
        kind="kde",
        col_wrap=3,
        height=3.5,
        facet_kws={"sharex": False, "sharey": False},
        common_norm=False,
    )

    if full_path:
        save_path = os.path.join(full_path, f"{title}_kde.svg")
        plt.savefig(save_path, format="svg")

    sns.displot(
        data=df,
        x=magnitude_column,
        col="Year",
        hue="Month",
        kind="ecdf",
        col_wrap=3,
        height=3.5,
        facet_kws={"sharex": False, "sharey": False},
    )

    if full_path:
        save_path = os.path.join(full_path, f"{title}_ecdf.svg")
        plt.savefig(save_path, format="svg")
